In [44]:
# -*- coding: utf-8 -*-
from __future__ import division      #除数可以显示为float
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import time                 #使用time stamp
import matplotlib.pyplot as plt
import math
import talib


from datetime import datetime
from datetime import timedelta

1、权益函数
v(z)=z^(a)  0<a<1    若z≥ 0
v(z)= -λ(-z)^β    λ>1 ,0<β<1   若z≤0
a，β约为0.88，λ约为2.25

2、加权函数
若z≥0，π(p)=  p^γ/[p^γ+(1-p)^γ ]^(1/γ)        其中γ>0     
若z<0，π(p)=  p^χ/[p^χ+(1-p)^χ ]^(1/χ)        其中χ>0
其中估计值为γ=0.61，χ=0.69

In [2]:
# 计算效用函数
# 输入函数：
# Z：价值变化的大小
# P：对应价值变化大小的概率
# 类型均为float函数
def Utility(Z,P):
    # 定义参数：
    # γ=0.61，χ=0.69
    a = 0.88
    b = 0.88
    c = 2.25
    d = 0.61
    e = 0.69

    # 第一步：计算V(z)，z是相应的价值变化
    # 第二步：通过概率计算加权函数
    if Z >= 0:
        Z_v = math.pow(Z, a)
        Z_w = math.pow(P, d) / math.pow((math.pow(P, d) + math.pow((1-P), d)),1/d)
    else:
        Z_v = (-c) * math.pow((-Z), b)
        Z_w = math.pow(P, e) / math.pow((math.pow(P, e) + math.pow((1-P), e)),1/e)

    # 第三部：反馈加权函数与价值函数的乘积
    return Z_v*Z_w

In [3]:
# print Utility(5000,0.001)
# print Utility(5,1)

print Utility(4000,0.2)
print Utility(3000,0.25)

# print Utility(5000,0.001)
# print Utility(5,1)

385.530787752
333.71511154


思路：
1. 过去15天周期的效用（价格+交易量），过去15天价格下跌得最惨的股票排序
2. 按照均值回归的原则，后15天会达到一个平衡（价格+交易量）
3. 假设交易量没有太大的波动；按照现在的价格、需达到平衡的价格、平衡价格对应的概率（P（X>P），假设正态分布，取过去60天左右的均值）之间应该有一个平衡
4. 概率高于一定时即是买入的时机（这个概率时机就是一个参数）
总而言之，概率大、效用高就是好

In [45]:
# 功能：计算过去一段时间合计的效用总额
# 输入参数：
# Input_Data_pd:必须包含price、volume、code三列
# Count_Period:计算的过去合计时间值
# Price_Current，目前的价格，用于计算价差

def Utility_Total(Input_Data_pd,Count_Period):
    
    # 对输入参数进行复制，处理
    Data_Total_pd = Input_Data_pd.copy()
    Data_Total_pd.index = range(0,Data_Total_pd.shape[0])
    Data_Total_pd['utility'] = np.nan
    
    # 针对不同的股票编码，生产相应的效用值
    stock_list = set(list(Data_Total_pd['code']))
    
    # 逐股票开始计算
    for stock_name in stock_list:
        # 暂时存储数据用，不同的股票编码都各有一个
        Data_temp_pd = pd.DataFrame()
        Data_temp_pd = Data_Total_pd[Data_Total_pd['code'] == stock_name]
        
        # 用于存储对应效用的数组
        utility_temp_list = []
        
        # 为了数组长度一致，将utility_temp_list中的前Count_Period个数组赋值
        for i in range(0,Count_Period):
            utility_temp_list.append(np.nan)
        
        # 逐行开始计算,从Count_Period行开始计算，一直到
        for row_i in range(Count_Period,Data_temp_pd.shape[0]):
            # utility_temp为暂存之前Count_Period的结果累加
            utility_temp = 0
            price_current = Data_temp_pd['price'].iloc[row_i]

            
            # 重复计算前15行的效用，累加之后赋值
            for i in range(1,Count_Period+1):
                price_pre = Data_temp_pd['price'].iloc[row_i-i]
                vol = Data_temp_pd['volume'].iloc[row_i-i]
                
                # 因为已经发生的情况，所以概率为始终1
                utility_temp += Utility(float(price_current-price_pre),1)*vol
            
            utility_temp_list.append(utility_temp)

        
        Data_Total_pd.loc[Data_Total_pd['code'] == stock_name,'utility'] = np.array(utility_temp_list)
    return Data_Total_pd
        

In [ ]:
# 功能：输入List数据，计算均值与方差
# 输入：
# input_data_list：输入的原始list数据
# n：计算有多少个均值
# 输出：
# ret_list_mean：输出均值数据
# ret_list_mean：输出方差数据

def Mean_Std_Generate(input_data_list,n):
    ret_mean_list = []
    ret_mean_list = []
    # 前n个
    for 

In [46]:
# 功能：输入Pandas数据，生成对应的指标数据
# 输入：
# input_data_pd：输入的原始Pandas数据
# 格式：必须包含的列包括：'code','date','close','high','low','volume','open'，用于计算指标
# 输出：
# ret_pd：输出Pandas数据
# ret_col：返回的特征向量
# 计算 15个指标，用于计算的指标包括：
# 'EMA_5', 'EMA_10','EMA_30','EMA_50','EMA_90'

def EMA_Generate(input_data_pd):
    
    ret_pd = pd.DataFrame()
    ret_pd = input_data_pd.copy()
    
    # 返回的特征向量
    ret_col = ['EMA_5', 'EMA_10','EMA_30','EMA_50','EMA_90']
    
    stock_list = set(list(input_data_pd['code']))
    
    # 计算相应的值
    for stock_name in stock_list:
 

        ################################   1、生成均线信息   ###################################################
        ret_pd["EMA_5"] = None
        ret_pd["EMA_10"] = None
        ret_pd["EMA_30"] = None
        ret_pd["EMA_50"] = None
        ret_pd["EMA_90"] = None
        
        for stock_name in stock_list:
            ema_5 = []
            ema_10 = []
            ema_30 = []
            ema_50 = []
            ema_90 = []            
            

            ema_price = ret_pd.loc[ret_pd['code'] == stock_name,"close"]
            ema_5 = talib.EMA(np.array(ema_price), timeperiod=5)
            ema_10 = talib.EMA(np.array(ema_price), timeperiod=10)
            ema_30 = talib.EMA(np.array(ema_price), timeperiod=30)
            ema_50 = talib.EMA(np.array(ema_price), timeperiod=50)
            ema_90 = talib.EMA(np.array(ema_price), timeperiod=90)


            ret_pd.loc[ret_pd['code'] == stock_name,"EMA_5"] = np.array(ema_5)
            ret_pd.loc[ret_pd['code'] == stock_name,"EMA_10"] = np.array(ema_10)
            ret_pd.loc[ret_pd['code'] == stock_name,"EMA_30"] = np.array(ema_30)
            ret_pd.loc[ret_pd['code'] == stock_name,"EMA_50"] = np.array(ema_50)
            ret_pd.loc[ret_pd['code'] == stock_name,"EMA_90"] = np.array(ema_90)

        ret_pd = ret_pd.fillna(0)
    
    return ret_pd,ret_col


In [47]:
Data_pd = pd.read_csv('price_list.csv')
Data_pd['price'] = Data_pd['close']
Period = 3 
Price_Current = 5.0

data = pd.DataFrame()
data = Utility_Total(Data_pd,Period)

EMA_Generate(data)

(            code        date  close   high    low     volume   open  price  \
 0    000001.XSHE   2017/9/12  11.54  11.54  11.27   84618342  11.38  11.54   
 1    000001.XSHE   2017/9/13  11.43  11.54  11.34   66823742  11.49  11.43   
 2    000001.XSHE   2017/9/14  11.32  11.59  11.24   88308777  11.43  11.32   
 3    000001.XSHE   2017/9/15  11.29  11.32  11.15   64609481  11.29  11.29   
 4    000001.XSHE   2017/9/18  11.25  11.32  11.20   60761296  11.25  11.25   
 5    000001.XSHE   2017/9/19  11.13  11.34  11.08   76421262  11.25  11.13   
 6    000001.XSHE   2017/9/20  11.29  11.37  11.05   78715469  11.14  11.29   
 7    000001.XSHE   2017/9/21  11.46  11.51  11.20   69240793  11.26  11.46   
 8    000001.XSHE   2017/9/22  11.44  11.52  11.31   59392762  11.43  11.44   
 9    000001.XSHE   2017/9/25  11.29  11.45  11.18   53239104  11.44  11.29   
 10   000001.XSHE   2017/9/26  11.05  11.30  10.96   96746090  11.26  11.05   
 11   000001.XSHE   2017/9/27  10.93  11.08  10.90  

尝试线性回归：
1. 对比当天收盘价与未来15天最高收盘价之间差异
2. 计算出增长率
3. 通过15天的效用，来进行线性拟合，看看效果